In [1]:
import numpy as np
import keras.api as keras
from keras.api.layers import Dense, Conv2D, Input, Lambda, UpSampling2D
from keras.api import Model
import tensorflow as tf
import matplotlib.pyplot as plt

2025-03-30 12:40:46.742369: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743349246.757754  118748 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743349246.762296  118748 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-30 12:40:46.778521: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [23]:
# Adaptive Instance Normalization (AdaIN)
def ada_in(x):
    style, content = x
    mean_s, var_s = tf.nn.moments(style, axes=[1, 2], keepdims=True)
    mean_c, var_c = tf.nn.moments(content, axes=[1, 2], keepdims=True)
    return (var_c**0.5) * ((content - mean_c) / (var_s**0.5)) + mean_s

# Mapping Network (f)
def mapping_network(latent_dim, w_dim):
    inputs = Input(shape=(latent_dim,))
    x = Dense(512, activation="relu")(inputs)
    for _ in range(7):
        x = Dense(512, activation="relu")(x)
    outputs = Dense(w_dim)(x)
    return Model(inputs, outputs, name="MappingNetwork")

# Synthesis Network (g)
def synthesis_network(w_dim, image_size=64):
    inputs = Input(shape=(w_dim,))
    
    # Inicializa um tensor constante (4x4x512)
    x = tf.Variable(tf.random.normal([1, 4, 4, 512]), trainable=True)
    
    # Primeira camada convolucional + AdaIN
    style = Dense(512)(inputs)
    x = Conv2D(512, (3,3), padding="same", activation="relu")(x)
    x = Lambda(ada_in, x[0].shape)([style, x])

    # Upsample + convolução em cascata
    for i in range(2, int(np.log2(image_size))):
        x = UpSampling2D()(x)
        x = Conv2D(512 // (2**i), (3,3), padding="same", activation="relu")(x)
        style = Dense(512 // (2**i))(inputs)
        x = Lambda(ada_in, x[0].shape)([style, x])
    
    # Camada final para gerar a imagem
    outputs = Conv2D(3, (1,1), activation="tanh")(x)
    
    return Model(inputs, outputs, name="SynthesisNetwork")

# Dimensões da rede
latent_dim = 128
w_dim = 512
image_size = 128

# Criando os modelos
mapping = mapping_network(latent_dim, w_dim)
synthesis = synthesis_network(w_dim, image_size)

# Pipeline completo do StyleGAN
latent_input = Input(shape=(latent_dim,))
w = mapping(latent_input)
image_output = synthesis(w)

stylegan = Model(latent_input, image_output, name="StyleGAN")
stylegan.summary()


Model: "StyleGAN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_50 (InputLayer)     │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ MappingNetwork (Functional)     │ (None, 512)            │     2,167,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SynthesisNetwork (Functional)   │ (None, 128, 128, 3)    │     1,077,899 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,245,195 (12.38 MB)

 Trainable params: 3,245,195 (12.38 MB)

 Non-trainable params: 0 (0.00 B)